In [3]:
# importing necessary libraries
import json
import math
import pandas as pd
import numpy as np
from itertools import combinations
from nltk.corpus import wordnet
from IPython.display import display

In [4]:
#preloading all schemas and categories
all_schemas={}
all_categories={}
all_tablenames={}
with open("final_schema.txt") as ip_file:
    for line in ip_file.readlines():
        json_obj=json.loads(line)
        all_schemas[json_obj["filename"]]=json_obj["schema"]
        all_categories[json_obj["filename"]]=json_obj["categories"]
        all_tablenames[json_obj["filename"]]=json_obj["tablename"]
        
#preloading the candidate keys
with open("Candidate_key_dict.txt",'r') as ip_file:
    cand_key=json.load(ip_file)

#preloading column and category similarity values of tables
with open("cos_similarity.txt",'r') as ip_file:
    json_object=json.load(ip_file)
col_sim = json_object["column_similarity"]
cat_sim = json_object["category_similarity"]

In [5]:
# all functions needed to generate ontologies
def get_synonyms(word):
    meanings=set()
    for synset in wordnet.synsets(word,pos=wordnet.NOUN):
        for lemma in synset.lemmas():
            meanings.add(lemma.name())
    for synset in wordnet.synsets(word,pos=wordnet.NOUN):
        for hypernym in synset.hypernyms():
            meanings.add(hypernym.lemma_names()[0])
    meanings.add(word)
    return list(meanings)

# takes input list and returns ontology as dictionary with every word in list as the key
def generate_list_ontology(list1):
    ontology={}
    for word in list1:
        ontology[word]=get_synonyms(word)
    return ontology

# takes the input as a schema and returns ontology for every column in the schema
def generate_schema_ontology(input_schema):
    ontology={}
    for col in input_schema:
        ontology[col]=get_synonyms(col)
    return ontology

In [6]:
# to generate cos similarity between two lists
def cos_sim(list1, list2):
    terms = set(list1).union(list2)
    intersect = set(list1) & set(list2)
    others = (set(list1)-intersect).union(set(list2)-intersect)
    product=0
    for word in terms:
        if word in intersect:
            product+=1
    l1mag = math.sqrt(len(list1))
    l2mag = math.sqrt(len(list2))
    if len(list1)==0 or len(list2)==0:
        return 0.0
    else:
        return product / (l1mag * l2mag)

In [41]:
# both the following functions
# col_only_list() and cat_and_col_list()
# functions return all the posssible tables that are suitable matches with input schema

# if input has only schema(columns and their dataypes)
# it returns a list of all tables with one or more columns that match with input schema(or its ontology)
def col_only_list(input_schema,input_sch_onto):
    possible_tables={}
    for file in all_schemas:
        schema=all_schemas[file]
        for col,d_type in schema.items():
            if (col in input_schema) and (input_schema[col]==d_type):
                if file in possible_tables:
                    possible_tables[file].append(col)
                else:
                    possible_tables[file]=[]
                    possible_tables[file].append(col)
            else:
                for a in input_sch_onto:
                    if (col in input_sch_onto[a]) and (input_schema[a]==d_type):
                        if file in possible_tables:
                            possible_tables[file].append(a)
                        else:
                            possible_tables[file]=[]
                            possible_tables[file].append(a)
    return possible_tables

# if input has categories as well as schema(columns and their dataypes)
# we consider it a match under the assumption that at least 75% category match exists
# then all tables that meet this criteria and have one or more columns that match are retured as a list 
def cat_and_col_list(input_categories,input_cat_onto,input_schema,input_sch_onto):
    possible_tables={}
    print("Category cosine similarity values that are greater than 75% :")
    print("File Name\t\tcategory cos similarity value")
    for file in all_categories:
        cat_list=[]
        category=all_categories[file]
        for cat in category:
            if cat in input_categories:
                cat_list.append(cat)
            else:
                for cat1 in input_cat_onto:
                    if cat in input_cat_onto[cat1]:
                        cat_list.append(cat1)
        cos_val=cos_sim(cat_list,input_categories)
        if cos_val > 0.75 :
            print(file,"\t\t",cos_val)
            schema=all_schemas[file]
            for col,d_type in schema.items():
                if (col in input_schema) and (input_schema[col]==d_type):
                    if file in possible_tables:
                        possible_tables[file].append(col)
                    else:
                        possible_tables[file]=[]
                        possible_tables[file].append(col)
                else:
                    for a in input_sch_onto:
                        if (col in input_sch_onto[a]) and (input_schema[a]==d_type):
                            if file in possible_tables:
                                possible_tables[file].append(a)
                            else:
                                possible_tables[file]=[]
                                possible_tables[file].append(a)
    print()
    return possible_tables

In [26]:
# function generates all possible combinations of list l taking elements n to 2 at a time and returns a dictionary
def generate_all_combinations(l):
    x={}
    a=len(l)
    for i in range(a,1,-1):
        x[i]=list(combinations(l,i))
    return x

# function generates all matching columns between the res_cols schema and columns of table in fname
def generate_matching_columns(res_cols,fname):
    a=all_schemas[fname]
    matching_columns={}
    res_onto=generate_schema_ontology(res_cols)
    for col , d_type in a.items():
        if (col in res_cols) and (d_type==res_cols[col]):
            matching_columns[col] = col
        else:
            for col_res in res_onto:
                if (col in res_onto[col_res]) and (d_type==res_cols[col_res]):
                    matching_columns[col] = col_res
                    break
    return matching_columns

# returns a merged table of all tables given in input list l
def merge_list(l):
    t1=pd.read_csv(l[0])
    t2=pd.read_csv(l[1])
    a=all_schemas[l[0]]
    b=all_schemas[l[1]]
    matching_columns={}
    a_onto=generate_schema_ontology(a)
    for col , d_type in b.items():
        if (col in a) and (d_type==a[col]):
            matching_columns[col] = col
        else:
            for col_a in a_onto:
                if (col in a_onto[col_a]) and (d_type==a[col_a]):
                    matching_columns[col] = col_a
                    break
    t2.rename(columns = matching_columns,inplace=True)
    res=t1.merge(t2,how='outer')
    for fname in l[2:]:
        d_types=[]
        res_cols={}
        for i in res.dtypes:
            d_types.append(str(i))
        for i,j in zip(res.columns,d_types):
            res_cols[i]=j
        matching_columns=generate_matching_columns(res_cols,fname)
        t=pd.read_csv(fname)
        t.rename(columns = matching_columns,inplace=True)
        res=res.merge(t,how='outer')
    return res

In [27]:
# metrics calculated :
# 1)nan_score(number of nulls in each column)
# 2)coverage_score(no of matching columns with input schema/total number of columns in input schema)
# 3)completeness_score(a combination of coverage and nan scores to determine how complete the result dataset is)

# nan score = {x : (no on nans in column/no of entries in column)} where x is each column in the table
# gives the nan score(no on nans/no of entries in table) for each column in the input table
def nan_score(table=-1,fname=-1):
    if fname!=-1:
        table=pd.read_csv(fname)
    nan_count={}
    a=len(table)
    for i in table.columns:
        x=a-table[i].count()
        s=str(x)+'/'+str(a)
        nan_count[i]=s
    return nan_count

#returns the coverage score and completeness score of a given table
#coverage score is calculated as : 
# coverage = (no of columns matching with input schema/total number of columns in input schema)
#completeness score is calculated as : 
# completeness = (sum(x*(non null entries)/(total entries in the column))/total number of columns in input schema) 
#  where x=1 if column present in input schema and x=0 if column is not present in the input schema
def coverage_and_completeness(table):
    ctr=0
    comp=0
    cols=table.columns
    l=len(table)
    for col in cols:
        if col in input_schema:
            ctr+=1
            comp+=1*(1-sum(pd.isnull(table[col]))/l)
    comp=comp/len(input_schema)
    cov=ctr/len(input_schema)
    return (cov,comp)

In [28]:
def ranking_display(comp_score,no_of_rows):
    print()
    comp_rank=sorted(comp_score,reverse=True)
    count=0
    l=sorted(no_of_rows, key=lambda k: no_of_rows[k],reverse=True)
    for i in comp_rank:
        if len(comp_score[i])==1:
            count+=1
            print("Rank "+str(count)+" : ",comp_score[i][0],"\t\tcompleteness score : ",i,"\t\tnumber of rows: ",no_of_rows[comp_score[i][0]])
        else:
            for j in l:
                if j in comp_score[i]:
                    count+=1
                    print("Rank "+str(count)+" : ",j,"\t\tcompleteness score : ",i,"\t\tnumber of rows: ",no_of_rows[j])
    print("\nRanking Complete!!\n")

In [29]:
#making a list of transformation functions
tf_onto={}
transform_funct_list={"int64":['average','sum','maximum','minimum','range','median','variance','standard deviation','mode','frequency','avg'],"float64":['average','sum','maximum','minimum','range','median','variance','standard deviation','mode','frequency','avg'],"object":["funct1","funct2","funct3"]}
for dtype,funct_list in transform_funct_list.items():
    tf_onto[dtype]=generate_list_ontology(transform_funct_list)

In [30]:
# takes output tables schema(columns and data_types) as the input, compare it with input schema and transformations required and
# returns a list of all the transformations applicable
def get_possible_transformations(cols):
    l={}
    for col,tran in transformations.items():
        if (col in cols):
            for i in tran:
                if i in transform_funct_list[input_schema[col]]:
                    if col in l:
                        l[col].append(i)
                    else:
                        l[col]=[]
                        l[col].append(i)
    return l

In [31]:
# this is a function to print the individual tables names, their nan score, columns that match with input schema, 
# coverage score, and completeness score along with possible transformations if any
def display_individual_matches(matching_tables,matching_tables_dict):
    global comp_score
    global no_of_rows
    f=open("demo/results.txt",'a')
    if transformations==-1:
        flag=0
    else:
        flag=1
    for i in matching_tables:
        print(i+"("+all_tablenames[i]+")",file=f)
        res=pd.read_csv(i)
        cov,comp=coverage_and_completeness(res)
        if flag==1:
            cols=res.columns
            l=get_possible_transformations(cols)
            print("possible transformations are : ",l,file=f)
            cov,comp=coverage_and_completeness(res)
        print('Missing Values(NANs score): ',nan_score(fname=i),file=f)
        print("Columns that match with input_schema:\n "+i+' : ',matching_tables_dict[i],file=f)
        print("Coverage Score : ",cov,"\t Completeness Score : ",comp,"\t Number of Rows : ",len(res),file=f)
        no_of_rows[i]=len(res)
        if comp in comp_score:
            comp_score[comp].append(i)
        else:
            comp_score[comp]=[]
            comp_score[comp].append(i)
        print(file=f)

In [40]:
# this functions calls the col_only_list() or cat_and_col_list() based on input query requirements
# it recieves a list of tbales with one or more columns matching with input schema and if categories involved then 75% category cos similarity with input schema
# then we check for 75% column cosine cimilarity score and return a list of all tables having 75% column cosine similarity score
def check_possible_matches():
    input_sch_onto=generate_schema_ontology(input_schema)
    if input_categories==-1:
        print('only schema')
        possible_tables=col_only_list(input_schema,input_sch_onto)
    else:
        print('category and schema\n')
        input_cat_onto=generate_list_ontology(input_categories)
        possible_tables=cat_and_col_list(input_categories,input_cat_onto,input_schema,input_sch_onto)
    matching_tables={}
    print("Columns cosine similarity values that are greater than 75% :")
    print("File Name\t\tcolumn cos similarity value")
    for i in possible_tables:
        cos_val=cos_sim(possible_tables[i],list(input_schema))
        if cos_val>0.75:
            print(i,"\t\t",cos_val)
            matching_tables[i]=possible_tables[i]
    print()
    return matching_tables

In [43]:
# the main function that is to be invoked and will call all the required functions to obtain the required matches and merges
# we have a list with all possible tables that are matches
# we generate all combinations of them in order to merge them
# a valid combination is one where every pair of tables have 50% cosine column and category similarity
# then we merge them and calculate the coverage score, null score and the completeness score
# all these details are displayed for each of the valid merges as well as individual tables
# the output is displayed in the output file "results.txt" along with the outputs in csv form
def get_matches():
    global comp_score
    global no_of_rows
    global ctr
    comp_score={}
    no_of_rows={}
    matching_tables_dict=check_possible_matches()
    matching_tables=list(matching_tables_dict)
    with open("demo/results.txt",'a') as f:
        print('******************************',file=f)
        print(file=f)
        print("All Possible Matches",file=f)
        print(file=f)
        print("Matching Tables : ",matching_tables,file=f)
        print("Matching Tables : ",matching_tables)
        for i in matching_tables:
            print(i,'\t',all_tablenames[i])
        if transformations==-1:
            print("\nNo Transformations In Input Schema",file=f)
            print(file=f)
        else:
            print('\nTransformations detected from input are : ',transformations,file=f)
            print(file=f)
    op_str1='demo/'
    op_str2='.csv'
    x=len(matching_tables)
    print(matching_tables)
    if x==0:
        with open("demo/results.txt",'a') as f:
            print("NO MATCHES FOUND",file=f)
            print(file=f)
    elif x==1:
        display_individual_matches(matching_tables,matching_tables_dict)
    elif x==2:
        a=matching_tables[0]+' : '+matching_tables[1]
        b=matching_tables[1]+' : '+matching_tables[0]
        if (a in cat_sim) or (b in cat_sim):
            if (cat_sim[a]>.50 and col_sim[a]>0.50) or (cat_sim[b]>.50 and col_sim[b]>0.50):
                ctr+=1
                res=res=merge_list([matching_tables[0],matching_tables[1]])
                op_string=op_str1+str(ctr)+op_str2
                res.to_csv(op_string,sep=',', index=False)
                cols=res.columns
                cov,comp=coverage_and_completeness(res)
                with open("demo/results.txt",'a') as f:
                    print(op_string,file=f)
                    if transformations!=-1:
                        l=get_possible_transformations(cols)
                        print("possible transformations are : ",l,file=f)
                    print('Missing Values(NANs): ',nan_score(table=res),file=f)
                    print("Columns that match with input_schema: ",file=f)
                    for j in matching_tables:
                        print(j+' : ',matching_tables_dict[j],file=f)
                    print("Coverage Score : ",cov,"\t Completeness Score : ",comp,"\t Number of Rows : ",len(res),file=f)
                    no_of_rows["output_"+str(ctr)+op_str2]=len(res)
                    if comp in comp_score:
                        comp_score[comp].append("output_"+str(ctr)+op_str2)
                    else:
                        comp_score[comp]=[]
                        comp_score[comp].append("output_"+str(ctr)+op_str2)
                    print(file=f)
        display_individual_matches(matching_tables,matching_tables_dict)
    else:
        count_comb=len(matching_tables)
        all_combos=generate_all_combinations(matching_tables)
        for i in range(count_comb,1,-1):
            for l in all_combos[i]:
                a=list(l)
                comb=list(combinations(a,2))
                flag=0
                for pair in comb:
                    if flag==0:
                        t1,t2=pair
                        if ((t1+' : '+t2) in cat_sim):
                            if (cat_sim[t1+' : '+t2]>=.50 and col_sim[t1+' : '+t2]>=0.50):
                                pass
                            else:
                                flag=1
                        else:
                            flag=1
                    else:
                        break
                if flag==0:
                    res=merge_list(a)
                    ctr+=1
                    op_string=op_str1+str(ctr)+op_str2
                    res.to_csv(op_string,sep=',', index=False)
                    cols=res.columns
                    cov,comp=coverage_and_completeness(res)
                    with open("demo/results.txt",'a') as f:
                        print(str(ctr)+op_str2+' : ',end='',file=f)
                        for j in a:
                            print(j+"("+all_tablenames[j]+")"+'\t\t',end='',file=f)
                        print(file=f)
                        if transformations!=-1:
                            l=get_possible_transformations(cols)
                            print("possible transformations are : ",l,file=f)
                        print('Missing Values(NANs): ',nan_score(table=res),file=f)
                        print("Columns that match with input_schema: ",file=f)
                        for j in a:
                            print(j+' : ',matching_tables_dict[j],file=f)
                        print("Coverage Score : ",cov,"\t Completeness Score : ",comp,"\t Number of Rows : ",len(res),file=f)
                        no_of_rows["output_"+str(ctr)+op_str2]=len(res)
                        if comp in comp_score:
                            comp_score[comp].append("output_"+str(ctr)+op_str2)
                        else:
                            comp_score[comp]=[]
                            comp_score[comp].append("output_"+str(ctr)+op_str2)
                        print(file=f)
        display_individual_matches(matching_tables,matching_tables_dict)
    with open("demo/results.txt",'a') as f:
        print('******************************',file=f)
    ranking_display(comp_score,no_of_rows)

In [48]:
ctr=0
comp_score={}
no_of_rows={}
with open("demo_inputs.txt") as ip_file:
    count=0
    for line in ip_file.readlines():
        ip_schema=json.loads(line)
        count+=1
        print("INPUT "+str(count))
        print(line)
        input_schema=ip_schema["schema"]
        if "categories" in ip_schema:
            input_categories=ip_schema["categories"]
        else:
            input_categories=-1
        if "transformations" in ip_schema:
            transformations=ip_schema["transformations"]
        else:
            transformations=-1
        with open("demo/results.txt",'a') as f:
            print("-------------------------------",file=f)
            print("INPUT "+str(count),file=f)
            print("Input Schema : ",file=f)
            print(line,file=f)
        get_matches()
        with open("demo/results.txt",'a') as f:
            print("-------------------------------",file=f)

INPUT 1
{"schema": {"Round": "int64", "Round.1": "object", "Circuit": "object", "Date": "object", "Pole Position": "object", "Fastest Lap": "object", "Winning Driver": "object"}}

only schema
Columns cosine similarity values that are greater than 75% :
File Name		column cos similarity value
203_408.csv 		 0.7559289460184544
203_514.csv 		 0.8451542547285165
204_253.csv 		 0.9258200997725514
204_40.csv 		 0.9258200997725514
204_455.csv 		 0.7559289460184544
204_569.csv 		 0.9999999999999999
204_63.csv 		 0.7559289460184544

Matching Tables :  ['203_408.csv', '203_514.csv', '204_253.csv', '204_40.csv', '204_455.csv', '204_569.csv', '204_63.csv']
203_408.csv 	 1989 Formula One season
203_514.csv 	 2008 Superleague Formula season
204_253.csv 	 1990 Superbike World Championship season
204_40.csv 	 2008 Superbike World Championship season
204_455.csv 	 1989 Formula One season
204_569.csv 	 1998 Swedish Touring Car Championship season
204_63.csv 	 2002 Italian Formula Three season
['203_408.c

In [54]:
input_schema={"Round": "int64", "Round.1": "object", "Circuit": "object", "Date": "object", "Pole Position": "object", "Fastest Lap": "object", "Winning Driver": "object"}
input_categories=-1
transformations=-1
comp_score={}
no_of_rows={}
matching_tables_dict=check_possible_matches()
matching_tables=list(matching_tables_dict)
print(matching_tables,matching_tables_dict)
display_individual_matches(matching_tables,matching_tables_dict)
ranking_display(comp_score,no_of_rows)

only schema
Columns cosine similarity values that are greater than 75% :
File Name		column cos similarity value
203_408.csv 		 0.7559289460184544
203_514.csv 		 0.8451542547285165
204_253.csv 		 0.9258200997725514
204_40.csv 		 0.9258200997725514
204_455.csv 		 0.7559289460184544
204_569.csv 		 0.9999999999999999
204_63.csv 		 0.7559289460184544

['203_408.csv', '203_514.csv', '204_253.csv', '204_40.csv', '204_455.csv', '204_569.csv', '204_63.csv'] {'203_408.csv': ['Date', 'Pole Position', 'Fastest Lap', 'Winning Driver'], '203_514.csv': ['Round', 'Round.1', 'Date', 'Pole Position', 'Fastest Lap'], '204_253.csv': ['Round', 'Round.1', 'Circuit', 'Date', 'Pole Position', 'Fastest Lap'], '204_40.csv': ['Round', 'Round.1', 'Circuit', 'Date', 'Pole Position', 'Fastest Lap'], '204_455.csv': ['Date', 'Pole Position', 'Fastest Lap', 'Winning Driver'], '204_569.csv': ['Round', 'Round.1', 'Circuit', 'Date', 'Pole Position', 'Fastest Lap', 'Winning Driver'], '204_63.csv': ['Round', 'Circuit', 'Da

In [55]:
input_schema={"Round": "int64", "Round.1": "object", "Circuit": "object", "Date": "object", "Pole Position": "object", "Fastest Lap": "object", "Winning Driver": "object"}

In [76]:
pd.read_csv("demo/12.csv")

,Round,Round.1,Circuit,Date,Pole Position,Fastest Lap,Winning Rider,Winning Driver,Winning Team,Winning Privateer,Winning driver,Winning team,Trophy winner
0,1,R1,Jerez,18 March,Raymond Roche,Stephane Mertens,Raymond Roche,NaN,NaN,NaN,NaN,NaN,NaN
1,1,R2,Jerez,18 March,Raymond Roche,Raymond Roche,Raymond Roche,NaN,NaN,NaN,NaN,NaN,NaN
2,2,R1,Donington,16 April,Giancarlo Falappa,Rob Phillis,Fred Merkel,NaN,NaN,NaN,NaN,NaN,NaN
3,2,R2,Donington,16 April,Giancarlo Falappa,Raymond Roche,Giancarlo Falappa,NaN,NaN,NaN,NaN,NaN,NaN
4,3,R1,Hungaroring,30 April,Malcolm Campbell,Raymond Roche,Fred Merkel,NaN,NaN,NaN,NaN,NaN,NaN
5,3,R2,Hungaroring,30 April,Malcolm Campbell,Fred Merkel,Raymond Roche,NaN,NaN,NaN,NaN,NaN,NaN
6,4,R1,Hockenheim,6 May,Raymond Roche,Fred Merkel,Fred Merkel,NaN,NaN,NaN,NaN,NaN,NaN
7,4,R2,Hockenheim,6 May,Raymond Roche,Raymond Roche,Stephane Mertens,NaN,NaN,NaN,NaN,NaN,NaN
8,5,R1,Mosport,3 June,Giancarlo Falappa,Raymond Roche,Raymond Roche,NaN,NaN,NaN,NaN,NaN,NaN
9,5,R2,Mosport,3 June,Giancarlo Falappa,Jamie James,Raymond Roche,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
d={"1":"2","2":"3",4:5,6:7}
e={"1":"2","2":"3",4:'5',6:7}

In [69]:
d==e

False

In [83]:
print("21.csv : 202_117.csv(German submarine U-559)   203_148.csv(German submarine U-9 (1935))   203_268.csv(German submarine U-502)\npossible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['substring']}\n22.csv : 202_117.csv(German submarine U-559)		203_148.csv(German submarine U-9 (1935))\npossible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['substring']}\n23.csv : 202_117.csv(German submarine U-559)		203_268.csv(German submarine U-502)\npossible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['substring']}\n24.csv : 203_148.csv(German submarine U-9 (1935))		203_268.csv(German submarine U-502)	\npossible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['funct1']}\n202_117.csv(German submarine U-559)\npossible transformations are :  {'Nationality': ['substring']}\n203_148.csv(German submarine U-9 (1935))\npossible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['funct1']}")

21.csv : 202_117.csv(German submarine U-559)   203_148.csv(German submarine U-9 (1935))   203_268.csv(German submarine U-502)
possible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['substring']}
22.csv : 202_117.csv(German submarine U-559)		203_148.csv(German submarine U-9 (1935))
possible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['substring']}
23.csv : 202_117.csv(German submarine U-559)		203_268.csv(German submarine U-502)
possible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['substring']}
24.csv : 203_148.csv(German submarine U-9 (1935))		203_268.csv(German submarine U-502)	
possible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['funct1']}
202_117.csv(German submarine U-559)
possible transformations are :  {'Nationality': ['substring']}
203_148.csv(German submarine U-9 (1935))
possible transformations are :  {'Tonnage (GRT)': ['sum', 'avg'], 'Nationality': ['funct1']}
